In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk

In [3]:
train = pd.read_excel('lastone.xlsx')                     #training data

In [6]:
train = train.drop('ID',axis=1)

In [7]:
train.head()

,Text,Polarity
0,"first think another Disney movie, might good, ...",1
1,"Put aside Dr. House repeat missed, Desperate H...",0
2,"big fan Stephen King's work, film made even gr...",1
3,watched horrid thing TV. Needless say one movi...,0
4,truly enjoyed film. acting terrific plot. Jeff...,1


In [8]:
train.shape

(25000, 2)

In [10]:
train['Text'].head()

0    first think another Disney movie, might good, ...
1    Put aside Dr. House repeat missed, Desperate H...
2    big fan Stephen King's work, film made even gr...
3    watched horrid thing TV. Needless say one movi...
4    truly enjoyed film. acting terrific plot. Jeff...
Name: Text, dtype: object

In [12]:
train['Text'][1]

'Put aside Dr. House repeat missed, Desperate Housewives (new) watch one. don\'t know exactly plagued movie. never thought I\'d say this, want 15 minutes fame back.<br /><br />Script, Direction, can\'t say. recognized stable actors (the usual suspects), thought Herbert Marshall class addition sat good cheesy flick. Boy, wrong. Dullsville.<br /><br />My favorite parts: "office girl" makes 029 keypunch puts cards 087 sorter. LOL @ "the computer". I\'d like someone identify next device - 477 ? It\'s even dinosaur\'s time.<br /><br />And dinosaurs don\'t much time waste.'

In [13]:
#removing html tags

review = []
for i in range(0,25000):
    r = re.sub('<br /><br />','', train['Text'][i])
    review.append(r)

In [14]:
review[1]

'Put aside Dr. House repeat missed, Desperate Housewives (new) watch one. don\'t know exactly plagued movie. never thought I\'d say this, want 15 minutes fame back.Script, Direction, can\'t say. recognized stable actors (the usual suspects), thought Herbert Marshall class addition sat good cheesy flick. Boy, wrong. Dullsville.My favorite parts: "office girl" makes 029 keypunch puts cards 087 sorter. LOL @ "the computer". I\'d like someone identify next device - 477 ? It\'s even dinosaur\'s time.And dinosaurs don\'t much time waste.'

In [15]:
#noise removing
#stemming for converting words to its root

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

corpus = []
for i in range(0,25000):
    r = re.sub('[^a-zA-Z]', ' ', review[i])
    r = r.lower().split()
    r = [word for word in r if word not in set(stopwords.words('english'))]
    r = [ps.stem(word) for word in r]
    r = ' '.join(r)
    corpus.append(r)

In [16]:
corpus[1]

'put asid dr hous repeat miss desper housew new watch one know exactli plagu movi never thought say want minut fame back script direct say recogn stabl actor usual suspect thought herbert marshal class addit sat good cheesi flick boy wrong dullsvil favorit part offic girl make keypunch put card sorter lol comput like someon identifi next devic even dinosaur time dinosaur much time wast'

In [18]:
#here i use lemmatizer for converting words to its root
# lemmatizer use grammer and context of the sentence for conversion

#difference between stemmer corpus[1] above and lemmatizer corpus_lm[1] below.

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
lm = WordNetLemmatizer()

corpus_lm = []
for i in range(0,25000):
    r = re.sub('[^a-zA-Z]', ' ', review[i])
    r = r.lower().split()
    r = [word for word in r if word not in set(stopwords.words('english'))]
    r = [lm.lemmatize(word) for word in r]
    r = ' '.join(r)
    corpus_lm.append(r)

In [19]:
corpus_lm[1]

'put aside dr house repeat missed desperate housewife new watch one know exactly plagued movie never thought say want minute fame back script direction say recognized stable actor usual suspect thought herbert marshall class addition sat good cheesy flick boy wrong dullsville favorite part office girl make keypunch put card sorter lol computer like someone identify next device even dinosaur time dinosaur much time waste'

In [20]:
from sklearn.feature_extraction.text import CountVectorizer        #bag of words

In [153]:
cv = CountVectorizer(0)

In [154]:
X = cv.fit_transform(corpus)

In [155]:
X

<25000x50566 sparse matrix of type '<class 'numpy.int64'>'
	with 2345437 stored elements in Compressed Sparse Row format>

In [172]:
cv1 = CountVectorizer(max_features=50560)

In [173]:
X1 = cv1.fit_transform(corpus_lm)

In [27]:
y= train['Polarity']

In [28]:
y.head()

0    1
1    0
2    1
3    0
4    1
Name: Polarity, dtype: int64

In [29]:
y.value_counts()

1    12500
0    12500
Name: Polarity, dtype: int64

In [32]:
from sklearn.feature_extraction.text import TfidfTransformer                      

In [174]:
tf = TfidfTransformer().fit(X1)

In [175]:
corp_tfidf = tf.transform(X1)

In [35]:
from sklearn.model_selection import train_test_split

In [176]:
X_train, X_test, y_train, y_test = train_test_split(corp_tfidf, y,test_size=0.3)

In [177]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB(alpha=1)

In [178]:
mnb.fit(X_train,y_train)

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)

In [179]:
predictions = mnb.predict(X_test)

In [180]:
from sklearn.metrics import confusion_matrix,classification_report

In [181]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.87      0.87      0.87      3793
          1       0.87      0.86      0.86      3707

avg / total       0.87      0.87      0.87      7500

